Instalar librerias

Conectar al dataset y descargar spider

In [104]:
import pandas as pd
import json
from google.colab import drive

# Montar Google Drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Probar los modelos

In [105]:
import json
import torch
import random
from transformers import BartForConditionalGeneration, BartTokenizer

# Rutas de los modelos y dataset
MODEL_PATH = "/content/drive/MyDrive/spider_models_fine_Tuned/BART-LARGE-NL2SQL"
SPIDER_DATASET_PATH = "/content/drive/MyDrive/spider/test.json"

In [108]:
# Cargar modelo y tokenizer
model = BartForConditionalGeneration.from_pretrained(MODEL_PATH)
tokenizer = BartTokenizer.from_pretrained(MODEL_PATH)
model.eval()

def generate_sql(question, db_id, schema):
    """Genera SQL a partir de una pregunta en lenguaje natural con información del esquema."""
    formatted_input = f"Database: {db_id} | Schema: {schema} | Question: {question}"
    inputs = tokenizer(formatted_input, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=128,
            num_beams=5,  # Evita exploraciones erróneas
            early_stopping=True,
            repetition_penalty=1.8,  # Ajustado para menor penalización
            length_penalty=2.0,  # Ajustado para fomentar estructuras más completas
            no_repeat_ngram_size=4,
            do_sample=False,  # Evita respuestas erráticas
            temperature=0.3  # Reducida para más determinismo
        )
    generated_sql = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Verificar si la salida es válida
    valid_keywords = ["SELECT", "FROM", "WHERE", "GROUP BY", "ORDER BY", "JOIN", "LIMIT"]
    if not any(keyword in generated_sql.upper() for keyword in valid_keywords):
        generated_sql = "INVALID SQL"

    return generated_sql

# Cargar el dataset de Spider
with open(SPIDER_DATASET_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

# Obtener el esquema de la base de datos real
def get_schema(db_id):
    """Extraer esquema real de la base de datos."""
    schema_dict = {
        "db1": "Tables: Rooms, Buildings | Columns: room_size, building_address",
        "db2": "Tables: Customers, Orders | Columns: customer_name, order_quantity, invoice_status",
        "db3": "Tables: Conference, Participants | Columns: conference_id, conference_name, year"
    }
    return schema_dict.get(db_id, "Schema not available")

# Seleccionar 5 preguntas aleatorias
random_samples = random.sample(data, 5)

In [111]:


# Evaluar para las preguntas aleatorias
results = []
for sample in random_samples:
    question = sample["question"]
    db_id = sample["db_id"]
    expected_sql = sample["query"]
    schema = get_schema(db_id)  # Obtener esquema de la base de datos
    generated_sql = generate_sql(question, db_id, schema)

    accuracy = 1 if generated_sql.strip().lower() == expected_sql.strip().lower() else 0
    results.append({
        "question": question,
        "expected_sql": expected_sql,
        "generated_sql": generated_sql,
        "accuracy": accuracy
    })

# Mostrar los resultados
for result in results:
    print("Pregunta:", result["question"])
    print("SQL Esperado:", result["expected_sql"])
    print("SQL Generado:", result["generated_sql"])
    print("Accuracy:", result["accuracy"])
    print("-" * 50)

# Calcular precisión global
overall_accuracy = sum(r["accuracy"] for r in results) / len(results)
print(f"Accuracy general: {overall_accuracy * 100:.2f}%")


Pregunta: Show all staff role codes and the number of document processes for each role.
SQL Esperado: SELECT staff_role_code ,  count(*) FROM Staff_in_processes GROUP BY staff_role_code
SQL Generado: SELECT T1.role_code,  count(*) FROM staff AS T1 JOIN cre_Doc_Workflow AS T2 ON T1.staff_role_code  =  T2.Staff_Roles_Code JOIN document_processes AS T3 ON T2.document_process_code!= T3.Document_Process_Code GROUP BY T1.Role_Code
Accuracy: 0
--------------------------------------------------
Pregunta: Count the number of different official languages.
SQL Esperado: SELECT count(DISTINCT language_id) FROM official_languages
SQL Generado: SELECT count(DISTINCT country_language) FROM country_language
Accuracy: 0
--------------------------------------------------
Pregunta: What is the total number of paintings exhibited in gallery 240?
SQL Esperado: SELECT count(*) FROM paintings WHERE LOCATION  =  "Gallery 240"
SQL Generado: SELECT count(*) FROM art_1 WHERE gallery  =  240;
Accuracy: 0
--------